In [ ]:
# pip install tiktoken
import pandas as pd 
# Set display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)
pd.set_option("display.float_format", "{:.4f}".format)

In [ ]:

# %pip install tiktoken tenacity tqdm google-generativeai
from google.cloud import bigquery
import pandas as pd
import json
import random
import time
import traceback
import os
from scipy.stats import linregress
from concurrent.futures import ThreadPoolExecutor, as_completed
from tenacity import retry, stop_after_attempt, wait_exponential
from tqdm import tqdm
import multiprocessing
import tiktoken
import google.generativeai as genai


In [ ]:
from google.cloud import bigquery
import pandas as pd
import json
import random
import time
import traceback
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv('/Users/rohankajgaonkar/projects/agents/.env')

from scipy.stats import linregress
from concurrent.futures import ThreadPoolExecutor, as_completed
import tiktoken
import google.generativeai as genai

# =========================================================
# Gemini configuration
# =========================================================
GEMINI_API_KEY = os.getenv('GOOGLE_API_KEY_2')
genai.configure(api_key=GEMINI_API_KEY)

GEMINI_MODEL = "models/gemini-2.5-flash"

# =========================================================
# BigQuery client
# =========================================================
client = bigquery.Client(project="masked")

In [ ]:

# =========================================================
# Processing Configuration
# =========================================================
MAX_WORKERS = min(32, multiprocessing.cpu_count() * 4)  # Dynamic scaling
SAMPLE_SIZE = 10  # Change to desired sample size
RANDOM_SEED = 12345

print(f"🔧 Configuration loaded:")
print(f"   - Max workers: {MAX_WORKERS}")
print(f"   - Model: {GEMINI_MODEL}")
print(f"   - Sample size: {SAMPLE_SIZE}")




In [ ]:

# =========================================================
# Token counting (monitoring only)
# =========================================================
_enc = tiktoken.get_encoding("cl100k_base")

def count_tokens(text: str) -> int:
    """Count tokens in text for cost estimation"""
    return len(_enc.encode(text))


In [ ]:

# =========================================================
# Gemini API with retry logic
# =========================================================
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=2, max=10),
    reraise=True
)
def query_gemini(
    prompt: str,
    model_name: str = GEMINI_MODEL,
    temperature: float = 0,
    max_output_tokens: int = 10248,
    timeout: int = 120
):
    """Query Gemini with automatic retry on failure"""
    model = genai.GenerativeModel(
        model_name,
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_output_tokens,
        }
    )

    try:
        response = model.generate_content(
            prompt,
            request_options={"timeout": timeout}
        )
        
        if not response or not response.text:
            raise ValueError("Empty response from Gemini")
            
        return response.text.strip()
        
    except Exception as e:
        print(f"❌ Gemini request failed: {e}")
        raise


In [ ]:
# =========================================================
# Trend helpers
# =========================================================
def classify_slope(slope, y):
    """Classify numeric slope into qualitative trend"""
    non_zero_count = (y != 0).sum()
    unique_vals = y.dropna().unique()

    if len(unique_vals) == 1 and unique_vals[0] == 0:
        return "no trend"
    if non_zero_count <= 1:
        return "no trend"
    if len(unique_vals) == 1:
        return "consistent"
    if slope < 0:
        return "downward trend"
    if slope > 0:
        return "upward trend"
    return "consistent"


def compute_trend(df, col):
    """Compute linear regression trend for a metric over time"""
    if col not in df.columns or "month_index" not in df.columns:
        return 0.0, "no trend"

    y = pd.to_numeric(df[col], errors="coerce")
    x = pd.to_numeric(df["month_index"], errors="coerce")

    mask = y.notna()
    x, y = x[mask], y[mask]

    if len(y) < 2:
        return 0.0, "no trend"

    if y.nunique() == 1 and y.iloc[0] == 0:
        return 0.0, "no trend"

    slope, *_ = linregress(x.values, y.values)
    return slope, classify_slope(slope, y)




In [ ]:

# =========================================================
# JSON serialization helper
# =========================================================
def serialize_for_json(obj):
    """Convert pandas/datetime objects to JSON-safe formats"""
    if isinstance(obj, dict):
        return {k: serialize_for_json(v) for k, v in obj.items()}
    if isinstance(obj, list):
        return [serialize_for_json(v) for v in obj]
    if hasattr(obj, "isoformat"):
        try:
            return obj.isoformat()
        except Exception:
            return str(obj)
    return obj

# Cell 8: Load Data Dictionary
# =========================================================
# Load data dictionary
# =========================================================
with open("/Users/rohankajgaonkar/Downloads/llm_data_dictionary.json", "r") as f:
    data_dict = json.load(f)

print(f"📚 Loaded data dictionary with {len(data_dict)} fields")


In [ ]:

# =========================================================
# Agent Archetypes
# =========================================================
archetypes = [
    {
        "archetype": "Newbie",
        "pain_points": [
            "Struggle to win first client trust",
            "Unsure which tools deliver results; risk of wasting spend",
            "Easily overwhelmed by many features and options",
            "Can get discouraged if results don't come quickly"
        ],
        "values": [
            "Affordable, low-risk way to start",
            "Simple, guided steps that show what to do next",
            "Quick visibility and credibility signals (Verified Listings, reviews)",
            "Early proof that actions lead to results"
        ],
        "PG_messaging_principles": [
            "Position PG as a safe and guided entry point to start their career",
            "Emphasise credibility and visibility from day one",
            "Show tools as simple helpers that build confidence step by step",
            "Reinforce early wins to boost momentum and confidence"
        ]
    },
    {
        "archetype": "Grind-to-Breakthrough",
        "pain_points": [
            "Work hard but see uneven results; credits burn quickly",
            "Lack scalable strategies; progress may feel slow",
            "Pressure to prove themselves with bigger wins"
        ],
        "values": [
            "Efficiency hacks to stretch credits and effort",
            "Clear ROI proof to justify spend",
            "Validation that they're on the right growth path",
            "Tools that feel like next-level upgrades"
        ],
        "PG_messaging_principles": [
            "Emphasise efficiency and ROI",
            "Highlight tools that provide a competitive edge over peers",
            "Reinforce that PG turns hard work into measurable progress"
        ]
    },
    {
        "archetype": "Grind-to-Sustain",
        "pain_points": [
            "Risk of burnout from heavy workload",
            "Time-poor; may be reluctant to adopt new tools",
            "Seeks stability, not risky experiments"
        ],
        "values": [
            "Reliable, repeatable lead flow",
            "Tools that save time and reduce admin effort",
            "Credibility signals that protect reputation"
        ],
        "PG_messaging_principles": [
            "Emphasise time-saving and lead gen features",
            "Show tools as reliable and easy to adopt into daily routines",
            "Reinforce PG's role in protecting income and maintaining edge"
        ]
    },
    {
        "archetype": "Driven Achiever",
        "pain_points": [
            "Highly competitive; want maximum visibility",
            "Pressure to maintain growth and status",
            "Need to scale across multiple listings or a team"
        ],
        "values": [
            "Top visibility and reach across all channels",
            "Recognition as industry leaders",
            "Premium tools that signal professionalism and success",
            "For team leads: tools that boost team efficiency and credibility"
        ],
        "PG_messaging_principles": [
            "Emphasise prestige and maximum exposure with premium tools",
            "Reinforce recognition and competitive advantage (awards, rankings, visibility)",
            "For team leads: highlight team enablement and scaling productivity"
        ]
    },
    {
        "archetype": "Industry Icon",
        "pain_points": [
            "Already credible; focus shifts to influence",
            "Desire to stay relevant at the top",
            "Need recognition beyond transactions (awards, prestige)"
        ],
        "values": [
            "Public recognition and prestige at the highest level",
            "Tools that showcase reputation and leadership",
            "Platforms that amplify voice and influence"
        ],
        "PG_messaging_principles": [
            "Emphasise prestige signals (awards, media, platform presence)",
            "Reinforce PG as the platform where icons are recognised"
        ]
    },
    {
        "archetype": "Part-Timer / Semi-Retired",
        "pain_points": [
            "Low or irregular activity; property work is secondary income or hobby",
            "Feels disconnected from market trends and younger peers",
            "Hesitant to invest time in learning complex features"
        ],
        "values": [
            "Flexibility to use only when needed, without pressure",
            "Credibility and trust signals that help maintain reputation despite low activity",
            "Simple reminders rather than hard-sell messaging"
        ],
        "PG_messaging_principles": [
            "Position PG as the easy, low-effort partner that keeps them credible and current",
            "Emphasise simplicity, flexibility, and time-saving convenience",
            "Use simple and low-pressure communication with no complexity or pushy messages"
        ]
    }
]

print(f" Loaded {len(archetypes)} agent archetypes")



In [ ]:
# =========================================================
# Query BigQuery for Agent Data For Renewals
# =========================================================

main_query = f"""
WITH agents_renewing_in_feb AS (
    -- First, find the unique agent_ids whose subscription ends in Feb 2026
    SELECT DISTINCT agent_id
    FROM `masked`
    WHERE subscription_end >= '2026-02-01' AND subscription_end < '2026-03-01'
)
-- Then, fetch all monthly data for that specific cohort of agents
SELECT T1.*
FROM `masked` AS T1
JOIN agents_renewing_in_feb AS T2 ON T1.agent_id = T2.agent_id
ORDER BY T1.agent_id, T1.data_month
"""

print("📥 Fetching data for the target agent cohort...")
agent_df_all = client.query(main_query).to_dataframe()

if agent_df_all.empty:
    print("No agents found with a subscription ending in February 2026. Please check the date range or data.")
else:
    print(f" Retrieved {len(agent_df_all):,} rows for {agent_df_all['agent_id'].nunique()} agents.")
    print(f"Date range of fetched data: {agent_df_all['data_month'].min()} to {agent_df_all['data_month'].max()}")


In [ ]:
agent_df_all['agent_id'].nunique() 

In [ ]:
agent_df_all[agent_df_all['agent_id'] == 624971]

In [ ]:
import os
file_path = "/Users/rohankajgaonkar/Downloads/llm_dictionary_l365.json"
print(f"File exists: {os.path.exists(file_path)}")

# List files in Downloads to see what's there
downloads_path = "/Users/rohankajgaonkar/Downloads"
json_files = [f for f in os.listdir(downloads_path) if f.endswith('.json')]
print(f"JSON files in Downloads: {json_files}")

In [ ]:
# import os
# file_path = "/Users/rohankajgaonkar/Downloads/llm_dictionary_l365.json"
# print(f"File exists: {os.path.exists(file_path)}")

# # List files in Downloads to see what's there
# downloads_path = "/Users/rohankajgaonkar/Downloads"
# json_files = [f for f in os.listdir(downloads_path) if f.endswith('.json')]
# print(f"JSON files in Downloads: {json_files}")

In [ ]:
# import matplotlib.pyplot as plt

# # Filter for the specific agent (assuming from cell 11)
# df = agent_df_all[agent_df_all['agent_id'] == 624971]

# fig, (ax1,ax2) = plt.subplots(1,2,sharex=True, sharey=True, figsize=(12, 3))
# # ax.plot(df['data_month'], df['activate_action'])
# # ax.set_title('Activate Action Over Time for Agent 624971')
# # ax.set_xlabel('Data Month')
# # ax.set_ylabel('Activate Action')
# ax1.plot(df['data_month'], df['activate_action'])
# ax2.plot(df['data_month'], df['repost_action'])
# ax1.set_title('Activate Action Over Time for Agent 624971')
# ax1.set_xlabel('Data Month')
# ax1.set_ylabel('Activate Action')
# ax2.set_title('Repost Action Over Time for Agent 624971')
# ax2.set_xlabel('Data Month')
# ax2.set_ylabel('reposts')

In [ ]:
# Get unique agents and their projected status
unique_agents_projected_status = agent_df_all.drop_duplicates(subset='agent_id')[['agent_id', 'firstname', 'lastname', 'projected_status_l365']]
unique_agents_projected_status['projected_status_l365'].value_counts()

In [ ]:
# =========================================================
#  Prompt Builder 
# =========================================================
def extract_relevant_fields(data_dict, agent_df):
    """Extract only field definitions that exist in agent data"""
    agent_columns = set(agent_df.columns)
    relevant_dict = {k: v for k, v in data_dict.items() if k in agent_columns}
    return json.dumps(relevant_dict, indent=2)


def build_prompt_for_single_agent(single_agent_df):
    """Build optimized prompt for single agent analysis"""
    
    # Subset columns
    cols = [
        'agent_id', 'data_month', 'firstname', 'lastname', 'age', 'active_days',
        'subscription_start', 'subscription_end', 'current_subscription',
        'credit_entitlement', 'credit_accumulation_cap', 'credit_purchase_cap',
        'loyalty_duration', 'estimated_earnings', 'total_listings',
        'sale_listings', 'rent_listings', 'roomrent_listings',
        'new_launch_condo_listings', 'new_launch_excondo_listings',
        'landed_sale_listings', 'landed_rent_listings', 'landed_roomrent_listings',
        'hdb_sale_listings', 'hdb_rent_listings', 'hdb_roomrent_listings',
        'condo_sale_listings', 'condo_rent_listings', 'condo_roomrent_listings',
        'com_sale_listings', 'com_rent_listings', 'listing_segment',
        'sub_ad_credit_consumed', 'disc_ad_credit_consumed',
        'landed_sale_transactions', 'hdb_sale_transactions',
        'condo_sale_transactions', 'landed_rent_transactions',
        'hdb_rent_transactions', 'condo_rent_transactions', 'leads', 'impressions',
        'monthly_sub_spend', 'monthly_disc_ac_spend', 'monthly_disc_pc_spend',
        'boost_incl_repost_action', 'turbo_incl_repost_action',
        'promote_list_action', 'leads_per_listing', 'impressions_per_listing',
        'archetype',
        'l365_total_listings', 'l365_ad_credit_consumed',
        'projected_tier_l365_ad_credit_consumed',
        'projected_tier_l365_com_listings',
        'projected_tier_l365_newlaunch_listings', 'l365_spend',
        'projected_tier_l365_spend_earnings_decile',
        'projected_subscription_l365', 'projected_status_l365',
        'l365_ad_credit_consumed_tier_decile', 'l365_spend_tier_decile',
        'projected_early_upgrade_potential_l365',
        'l30_total_listings', 'projected_churn_potential_updated',
        'turbopro_incl_repost_action'
    ]
    cols_present = [c for c in cols if c in single_agent_df.columns]
    agent_df = single_agent_df[cols_present].copy()

    # Convert dates
    for dt_col in ["data_month", "subscription_start", "subscription_end"]:
        if dt_col in agent_df.columns:
            agent_df[dt_col] = pd.to_datetime(agent_df[dt_col], errors="coerce")

    # Fill numeric columns
    numeric_cols = agent_df.select_dtypes(include=["number"]).columns
    agent_df[numeric_cols] = agent_df[numeric_cols].fillna(0)

    # Agent details
    details_cols = [
        'agent_id', 'firstname', 'lastname',
        'subscription_start', 'subscription_end', 'current_subscription',
        'credit_entitlement', 'credit_accumulation_cap', 'credit_purchase_cap',
        'listing_segment', 'archetype', 'active_days'
    ]
    details_cols = [c for c in details_cols if c in agent_df.columns]
    agent_details = agent_df[details_cols].drop_duplicates().to_dict(orient="records")[0]

    # Archetype matching
    archetype_name = agent_details.get("archetype")
    agent_archetype_json = next(
        (a for a in archetypes if a["archetype"] == archetype_name),
        {}
    )
    agent_archetype_json_str = json.dumps(agent_archetype_json, indent=2)

    # Calculate aggregates
    tx_cols = [
        'landed_sale_transactions', 'hdb_sale_transactions', 'condo_sale_transactions',
        'landed_rent_transactions', 'hdb_rent_transactions', 'condo_rent_transactions'
    ]
    agent_df["total_transactions"] = agent_df[[c for c in tx_cols if c in agent_df.columns]].sum(axis=1, skipna=True)

    credit_cols = ['sub_ad_credit_consumed', 'disc_ad_credit_consumed']
    agent_df["total_credit_consumption"] = agent_df[[c for c in credit_cols if c in agent_df.columns]].sum(axis=1, skipna=True)

    # Monthly data
    monthly_cols = [
        'data_month', 'total_listings', 'total_credit_consumption',
        'total_transactions', 'leads', 'impressions',
        'boost_incl_repost_action', 'turbo_incl_repost_action',
        'turbopro_incl_repost_action'
    ]
    monthly_cols = [c for c in monthly_cols if c in agent_df.columns]
    agent_df_monthly = agent_df[monthly_cols].copy().sort_values("data_month")
    agent_df_monthly["month_index"] = range(len(agent_df_monthly))

    # Ensure numeric
    for c in ['total_listings', 'total_credit_consumption', 'total_transactions',
              'leads', 'impressions', 'boost_incl_repost_action',
              'turbo_incl_repost_action', 'turbopro_incl_repost_action']:
        if c in agent_df_monthly.columns:
            agent_df_monthly[c] = pd.to_numeric(agent_df_monthly[c], errors="coerce").fillna(0)

    # Compute trends
    total_credit_consumption_slope, total_credit_consumption_trend = compute_trend(agent_df_monthly, "total_credit_consumption")
    total_transactions_slope, total_transactions_trend = compute_trend(agent_df_monthly, "total_transactions")
    leads_slope, leads_trend = compute_trend(agent_df_monthly, "leads")
    impressions_slope, impressions_trend = compute_trend(agent_df_monthly, "impressions")
    total_listings_slope, total_listings_trend = compute_trend(agent_df_monthly, "total_listings")
    boost_incl_repost_action_slope, boost_incl_repost_action_trend = compute_trend(agent_df_monthly, "boost_incl_repost_action")
    turbo_incl_repost_action_slope, turbo_incl_repost_action_trend = compute_trend(agent_df_monthly, "turbo_incl_repost_action")

    # L365 data
    l365_cols = [
        'l365_total_listings', 'l365_ad_credit_consumed',
        'projected_tier_l365_ad_credit_consumed',
        'projected_tier_l365_com_listings',
        'projected_tier_l365_newlaunch_listings', 'l365_spend',
        'projected_tier_l365_spend_earnings_decile',
        'projected_subscription_l365', 'projected_status_l365',
        'l365_ad_credit_consumed_tier_decile', 'l365_spend_tier_decile',
        'projected_early_upgrade_potential_l365',
        'l30_total_listings', 'projected_churn_potential_updated'
    ]
    l365_cols = [c for c in l365_cols if c in agent_df.columns]
    agent_l365 = agent_df[l365_cols].drop_duplicates().to_dict(orient="records")[0] if l365_cols else {}
    
    # # *** NEW addtiona by RDK: Extract renewal status for the prompt *** if code fails remover
    # projected_renewal_status = agent_l365.get("projected_status_l365", "Unknown")

    # Serialize to JSON
    agent_details_json = serialize_for_json(agent_details)
    agent_l365_json = serialize_for_json(agent_l365)
    
    # Extract only relevant field definitions (OPTIMIZATION)
    relevant_data_dict_str = extract_relevant_fields(data_dict, agent_df)

    agent_details_str = json.dumps(agent_details_json, indent=2)
    agent_l365_str = json.dumps(agent_l365_json, indent=2)

    # Build optimized prompt
    prompt = f"""
====================
System
====================
Act like a veteran Sales Manager and Data Analyst specializing in agent performance and health analytics for Singapore property marketplace listings.
You are coaching a Sales Person through an evidence-based review of a single agent's performance.
Your job: Interpret the agent's behavior, assess their overall account health (growth, stability, or decline), and propose targeted engagement actions using ONLY the provided data dictionary and agent datasets.

====================
Inputs
====================
1. Data Dictionary (Relevant Fields Only): {relevant_data_dict_str}
2. Agent details: {agent_details_str}
3. Agent data for last 365 days: {agent_l365_str}
4. Archetype Profile: {agent_archetype_json_str}

Archetype Components:
- Pain Points: Main challenges the agent experiences. Use this to interpret trends and understand barriers to engagement.
- What They Value: Motivations and priorities. Use this to frame recommendations that align with their goals.
- PG Messaging Principles: Best communication approaches. Use this to craft interventions aligned with their style.

====================
Objective
====================
1. Describe what's happening with this agent based on data trends and context
2. Assess overall account health: Is the agent growing, stable, at-risk, or churning?
3. Deliver concrete, prioritized actions the sales manager can execute now
4. Hard limit: Section 1 ≤250 words, Section 2 ≤150 words

====================
Guardrails
====================
1. DO NOT start the output with any conversational text, preamble, or introduction like "Alright team..." or "Here's a review...". The output must begin directly with the "1. Summary..." header.
2. Do not fabricate metrics not derivable from the inputs.
3. Reference fields exactly as named in the data dictionary.
4. Never infer peer benchmarks unless provided via "tier_decile" fields. Top 40% = 40th percentile within tier.
5. Ignore fields in data dictionary that are not in datasets.
6. Agents in Bronze, Silver, Silver Plus, Advance, and Standard tiers CANNOT use "Turbo Pro" actions. Do not evaluate them on this.
7. Describe their last-365-days ad credit consumption decile vs peers in their tier (do not print the field name).
8. Never print slope values. Only state: upward trend / downward trend / consistent / no trend.

====================
KEY HEALTH SIGNALS TO ASSESS
====================
I provide trend analysis variables from regression slopes across 7 agent activity metrics over 12 months.

**Trend Interpretation:**
- **Upward trend**: Strong engagement, growth opportunity, potential for upsell
- **Downward trend**: Declining engagement, churn risk, needs intervention
- **Consistent**: Stable performance, maintain relationship
- **No trend**: Insufficient data or inactive

=== Metric Definitions ===
total_credit_consumption: Advertising spend. Upward = increasing investment. Downward = disengagement or budget cuts.
total_transactions: Closed deals. Upward = business success. Downward = struggling pipeline.
leads: Inquiry volume. Upward = strong listing performance. Downward = weakening visibility.
impressions: Platform visibility. Upward = active promotion. Downward = reduced activity.
total_listings: Inventory count. Upward = expanding business. Downward = shrinking portfolio.
boost_incl_repost_action: Basic promotion usage. Upward = increasing effort. Downward = reduced belief in tools.
turbo_incl_repost_action: Premium promotion usage. Upward = premium engagement. Downward = cost-cutting.

=== Trend Variables ===
total_credit_consumption: {total_credit_consumption_trend}
total_transactions: {total_transactions_trend}
leads: {leads_trend}
impressions: {impressions_trend}
total_listings: {total_listings_trend}
boost_incl_repost_action: {boost_incl_repost_action_trend}
turbo_incl_repost_action: {turbo_incl_repost_action_trend}

=== Analysis Instructions ===
1. Identify upward, downward, consistent, and no-trend metrics.
2. Assess overall health: Growing (multiple upward trends) / Stable (consistent) / At-Risk (some downward) / Churning (multiple downward).
3. For new agents (active_days < 270): Note limited data and lower confidence in trend assessment.
4. Context matters: Interpret trends through archetype pain points and values.
5. If no specific growth or recovery actions are needed (e.g., for a stable agent), recommend a simple relationship-building check-in.


====================
OUTPUT SPECIFICATION
====================
Output EXACTLY two sections with the headers "1. Summary of Agent Health & Behaviour**" and "2. Recommended Engagement Actions". DO NOT omit the second section for any reason.

**1. Summary of Agent Health & Behaviour**
- State: firstname, lastname, archetype, listing segment, current subscription tier
- Describe their performance vs tier peers using l365_ad_credit_consumed_tier_decile
- Summarize trend patterns: Which metrics are growing, stable, or declining?
- Assess overall health status: Growing / Stable / At-Risk / Churning
- Interpret trends in context of their archetype pain points and values
- Do NOT print slope numbers or field names

**2. Recommended Engagement Actions [Beta]**
- Provide 2-3 concrete, prioritized actions the sales manager can execute now
- For growing agents: Upsell opportunities, recognition, expansion strategies
- For stable agents: Retention tactics, loyalty rewards, efficiency tools
- For at-risk agents: Re-engagement, support, win-back incentives
- For churning agents: Urgent intervention, root cause discovery, rescue offers
- Include 1 sentence that mirrors PG messaging principle phrasing
- Tailor to agent's ambition level (breaking through / sustaining / achieving icon status)

====================
VALIDATION CHECKLIST (DO NOT OUTPUT)
====================
✓ Used data dictionary to define every metric?
✓ Avoided printing column names?
✓ Avoided penalizing Turbo Pro nulls for Bronze/Silver/Silver Plus/Advance/Standard?
✓ Output limited to two sections only?
✓ No fabricated metrics?
✓ Assessed overall health, not just churn?
"""
    
    return prompt, agent_details




In [ ]:

# =========================================================
# Single Agent Processor with Error Handling
# =========================================================
def process_single_agent(agent_id, df_agent):
    """Process one agent with retry logic and error handling"""
    try:
        prompt, details = build_prompt_for_single_agent(df_agent)
        prompt_tokens = count_tokens(prompt)
        
        print(f" Agent {agent_id} | tokens={prompt_tokens:,}")
        
        llm_output = query_gemini(
            prompt,
            max_output_tokens=10248,
            timeout=90
        )
        
        if not llm_output:
            print(f"Empty response for agent {agent_id}")
            return None
        
        return {
            "agent_id": agent_id,
            "firstname": details.get("firstname"),
            "lastname": details.get("lastname"),
            "archetype": details.get("archetype"),
            "current_subscription": details.get("current_subscription"),
            "listing_segment": details.get("listing_segment"),
            "active_days": details.get("active_days"),
            "summary": llm_output,
            "prompt_tokens": prompt_tokens
        }
    
    except Exception as e:
        print(f"❌ Agent {agent_id}: {str(e)}")
        return {
            "agent_id": agent_id,
            "error": str(e),
            "firstname": None,
            "lastname": None,
            "summary": None
        }



In [ ]:

# =========================================================
# Run Parallel Processing with Progress Tracking
# =========================================================
print(f"\n Starting parallel processing with {MAX_WORKERS} workers...\n")

start_time = time.time()
results = []

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # Submit all tasks
    futures = {
        executor.submit(process_single_agent, agent_id, df_agent): agent_id
        for agent_id, df_agent in agent_df_all.groupby("agent_id")
    }
    
    # Progress bar
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing agents"):
        res = future.result()
        if res:
            results.append(res)

# Create results dataframe
results_df = pd.DataFrame(results)

# Separate successful vs failed
successful_df = results_df[results_df['summary'].notna()].copy()
failed_df = results_df[results_df['summary'].isna()].copy()

elapsed = time.time() - start_time


# print(f"✅ Successful: {len(successful_df)}/{len(results_df)}")
# print(f"❌ Failed: {len(failed_df)}")
print(f"Total time: {elapsed:.1f}s (~{elapsed/60:.2f} minutes)")
print(f"⚡ Avg time per agent: {elapsed/len(results_df):.2f}s")

if len(successful_df) > 0:
    avg_tokens = successful_df['prompt_tokens'].mean()
    total_tokens = successful_df['prompt_tokens'].sum()
    print(f"Avg prompt tokens: {avg_tokens:,.0f}")
    print(f"Total tokens: {total_tokens:,.0f}")
    print(f"Est. cost (Flash): ${total_tokens * 0.000001:.4f}")

print(f"{'='*60}\n")




In [ ]:
if len(successful_df) > 0:
    avg_tokens = successful_df['prompt_tokens'].mean()
    total_tokens = successful_df['prompt_tokens'].sum()
    print(f"🎫 Avg prompt tokens: {avg_tokens:,.0f}")
    print(f"🎫 Total tokens: {total_tokens:,.0f}")
    print(f"💰 Est. cost (Flash): ${total_tokens * 0.000001:.4f}")

In [ ]:
successful_df.head()

In [ ]:
for index, row in results_df.iterrows():
    print(f"Agent: {row['firstname']} {row['lastname']}")
    print(row['summary'])
    print("\n" + "="*50 + "\n")




In [ ]:
# =========================================================
# Analyze Results Distribution
# =========================================================
if len(successful_df) > 0:
    print("\n📊 RESULTS ANALYSIS\n")
    
    print("By Archetype:")
    print(successful_df['archetype'].value_counts())
    
    print("\nBy Subscription Tier:")
    print(successful_df['current_subscription'].value_counts())
    
    print("\nBy Listing Segment:")
    print(successful_df['listing_segment'].value_counts())
    
    # Token usage distribution
    print("\nToken Usage Statistics:")
    print(successful_df['prompt_tokens'].describe())

In [ ]:
import os

# Define the save directory
save_dir = "/Users/rohankajgaonkar/Downloads"

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Generate timestamp
timestamp = pd.Timestamp.now().strftime("%Y%m%d")

if len(successful_df) > 0:
    output_file = os.path.join(save_dir, f"agent_health_summaries_{timestamp}.csv")
    successful_df.to_csv(output_file, index=False)
    print(f"💾 Saved successful results to: {output_file}")

if len(failed_df) > 0:
    error_file = os.path.join(save_dir, f"failed_agents_{timestamp}.csv")
    failed_df.to_csv(error_file, index=False)
    print(f"⚠️  Saved failed agents to: {error_file}")